# Double-bracket Iteration other cost functions and respective scheduling

This notebook presents two additional cost functions for the double-bracket flow: least-squares and energy fluctuation with their respectice scheduling methods.

In [ ]:
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

from qibo import hamiltonians, set_backend
from qibo.models.dbi.double_bracket import DoubleBracketGeneratorType, DoubleBracketScheduling, DoubleBracketIteration, DoubleBracketCostFunction
from qibo.models.dbi.utils import *
from qibo.models.dbi.utils_scheduling import *

### Least-squares

The cost function is defined as: $\frac{1}{2}||D-H_k||^2 =\frac{1}{2}(||D||^2+||H||^2) -Tr(D H_k)$ as in https://epubs.siam.org/doi/abs/10.1137/S0036141092229732?journalCode=sjmael. We seek to maximize this function at each iteration.

In [ ]:
# Hamiltonian
set_backend("qibojit", platform="numba")

# hamiltonian parameters
nqubits = 5
h = 3.0

# define the hamiltonian
H_TFIM = hamiltonians.TFIM(nqubits=nqubits, h=h)

# define the least-squares cost function
cost = DoubleBracketCostFunction.least_squares
# initialize class
dbi = DoubleBracketIteration(deepcopy(H_TFIM),mode=DoubleBracketGeneratorType.single_commutator,cost=cost)



In [ ]:
# generate data for plotting sigma decrease of the first step
d = np.diag(np.linspace(1,2**nqubits,2**nqubits))
s_space = np.linspace(1e-5, 0.6, 1000)
off_diagonal_norm_diff = []
potential = []
for s in s_space:
    dbi_eval = deepcopy(dbi)
    dbi_eval(s,d=d)
    off_diagonal_norm_diff.append(dbi_eval.off_diagonal_norm - dbi.off_diagonal_norm)
    potential.append(dbi_eval.least_squares(D=d))

# grid_search
step_grid = dbi.choose_step(scheduling=DoubleBracketScheduling.grid_search,d=d)
print('grid_search step:', step_grid)
# hyperopt
step_hyperopt = dbi.choose_step(scheduling=DoubleBracketScheduling.hyperopt,d=d, max_evals=100, step_max=0.6)
print('hyperopt_search step:', step_hyperopt)
# polynomial
step_poly = dbi.choose_step(scheduling=DoubleBracketScheduling.polynomial_approximation,d=d, n=3)
print('polynomial_approximation step:', step_poly)

In [ ]:
# Plot the results
plt.figure()
plt.plot(s_space, potential)
plt.xlabel('s')
plt.axvline(x=step_grid, color='r', linestyle='-',label='grid_search')
plt.axvline(x=step_hyperopt, color='g', linestyle='--',label='hyperopt')
plt.axvline(x=step_poly, color='m', linestyle='-.',label='polynomial')
plt.title('First DBI step')
plt.ylabel('Least squares cost function')
plt.legend()
plt.figure()
plt.plot(s_space, off_diagonal_norm_diff)
plt.axvline(x=step_grid, color='r', linestyle='-',label='grid_search')
plt.axvline(x=step_hyperopt, color='g', linestyle='--',label='hyperopt')
plt.axvline(x=step_poly, color='m', linestyle='-.',label='polynomial')
plt.ylabel(r'$||\sigma(H_0)||-\sigma(H_k)||$')
plt.xlabel('s')
plt.title('First DBI step')
plt.legend()
print('The minimum for cost function in the tested range is:', step_grid)

Comparison of the least-squares cost function with the original cost function using the polynomial scheduling method

In [ ]:
d = np.diag(np.linspace(1,2**nqubits,2**nqubits))
off_diagonal_norm_diff = [dbi.off_diagonal_norm]
off_diagonal_norm_diff_least_squares = [dbi.off_diagonal_norm]
iters = 100
dbi_ls = deepcopy(dbi)
cost = DoubleBracketCostFunction.off_diagonal_norm
dbi_od = DoubleBracketIteration(deepcopy(H_TFIM),mode=DoubleBracketGeneratorType.single_commutator,cost=cost)
for _ in range(iters):
    step_poly = dbi_od.choose_step(scheduling=DoubleBracketScheduling.polynomial_approximation, d=d, n=3)
    dbi_od(step_poly,d=d)
    step_poly = dbi_ls.choose_step(scheduling=DoubleBracketScheduling.polynomial_approximation, d=d, n=3)
    dbi_ls(step_poly,d=d)
    off_diagonal_norm_diff.append(dbi_od.off_diagonal_norm)
    off_diagonal_norm_diff_least_squares.append(dbi_ls.off_diagonal_norm)

In [ ]:
plt.figure()
plt.plot(range(iters+1), off_diagonal_norm_diff, label=r'Off-diagonal norm')
plt.plot(range(iters+1), off_diagonal_norm_diff_least_squares, label=r'Least squares')
plt.xlabel('Iterations')
plt.ylabel(r'$||\sigma(H_k)||$')
plt.legend()


### Energy fluctuation

This cost function is defined as: $\Xi_k^2 (\mu) = \langle \mu | H_k^2| \mu \rangle - \langle \mu | H_k| \mu \rangle^2$

In [ ]:
# Hamiltonian
set_backend("qibojit", platform="numba")

# hamiltonian parameters
nqubits = 3
h = 3.0

# define the hamiltonian
H_TFIM = hamiltonians.TFIM(nqubits=nqubits, h=h)

# define the energy fluctuation cost function
cost = DoubleBracketCostFunction.energy_fluctuation
# define the state
state = 0
# initialize class
dbi = DoubleBracketIteration(deepcopy(H_TFIM),mode=DoubleBracketGeneratorType.single_commutator,cost=cost, state=state)

In [ ]:
# generate data for plotting sigma decrease of the first step
d = np.diag(np.linspace(1,2**nqubits,2**nqubits))
s_space = np.linspace(1e-5, 0.9, 1000)
off_diagonal_norm_diff = []
fluctuation = []
for s in s_space:
    dbi_eval = deepcopy(dbi)
    dbi_eval(s,d=d)
    off_diagonal_norm_diff.append(dbi_eval.off_diagonal_norm - dbi.off_diagonal_norm)
    fluctuation.append(dbi_eval.energy_fluctuation(state=state))

# grid_search
step_grid = dbi.choose_step(scheduling=DoubleBracketScheduling.grid_search,d=d)
print('grid_search step:', step_grid)
# hyperopt
step_hyperopt = dbi.choose_step(scheduling=DoubleBracketScheduling.hyperopt,d=d, max_evals=100, step_max=0.6)
print('hyperopt_search step:', step_hyperopt)
# polynomial
step_poly = dbi.choose_step(scheduling=DoubleBracketScheduling.polynomial_approximation,d=d, n=3)
print('polynomial_approximation step:', step_poly)

In [ ]:
# Plot the results
plt.figure()
plt.plot(s_space, fluctuation)
plt.xlabel('s')
plt.axvline(x=step_grid, color='r', linestyle='-',label='grid_search')
plt.axvline(x=step_hyperopt, color='g', linestyle='--',label ='hyperopt')
plt.axvline(x=step_poly, color='m', linestyle='-.',label='polynomial')
plt.title('First DBI step')
plt.ylabel('Energy fluctuation')
plt.legend()
plt.figure()
plt.plot(s_space, off_diagonal_norm_diff)
plt.axvline(x=step_grid, color='r', linestyle='-',label='grid_search')
plt.axvline(x=step_hyperopt, color='g', linestyle='--',label='hyperopt')
plt.axvline(x=step_poly, color='m', linestyle='-.',label='polynomial')
plt.ylabel(r'$||\sigma(H_0)||-\sigma(H_k)||$')
plt.xlabel('s')
plt.title('First DBI step')
plt.legend()
print('The minimum for cost function in the tested range is:', step_grid)

In [ ]:
d = np.diag(np.linspace(1,2**nqubits,2**nqubits))
off_diagonal_norm_diff = [dbi.off_diagonal_norm]
energy_fluc = [dbi.energy_fluctuation(state=state)]
iters = 50
dbi_ = deepcopy(dbi)
for _ in range(iters):
    step_poly = dbi_.choose_step(scheduling=DoubleBracketScheduling.polynomial_approximation, d=d, n=3)
    dbi_(step_poly,d=d)
    off_diagonal_norm_diff.append(dbi_.off_diagonal_norm)
    energy_fluc.append(dbi_.energy_fluctuation(state=state))

In [ ]:
plt.figure()
plt.plot(range(iters+1), off_diagonal_norm_diff)
plt.xlabel('Iterations')
plt.ylabel(r'$||\sigma(H_k)||$')

plt.figure()
plt.plot(range(iters+1), energy_fluc)
plt.xlabel('Iterations')
plt.ylabel(r'Energy fluctuation')


In [ ]:
iters = 10
columnNorm = np.empty((2**nqubits,iters))
d = (np.diag(np.linspace(1,2**nqubits,2**nqubits)))
for i in range(2**nqubits):
    dbi_ = deepcopy(dbi)
    dbi_.state = i
    for j in range(iters):
        step_poly = dbi_.choose_step(scheduling=DoubleBracketScheduling.polynomial_approximation, d=d, n=3)
        dbi_(step_poly,d=d)
        columnNorm[i,j] = np.linalg.norm(dbi_.h.matrix[:,i])
    

In [ ]:
eigvals = np.linalg.eigh(dbi_.h.matrix)[0]
plt.figure()
for i in range(2**nqubits):
    plt.plot(range(iters), columnNorm[i], label='State ' + str(i))
    plt.axhline(y=eigvals[i], color='r', linestyle='--')
plt.xlabel('Iterations')


In [ ]:
cost = DoubleBracketCostFunction.least_squares
dbi = DoubleBracketIteration(deepcopy(H_TFIM),mode=DoubleBracketGeneratorType.single_commutator,cost=cost)
d = np.diag(np.linspace(1,2**nqubits,2**nqubits))

step = 1e-2
iterations = 100

d, loss, grad, diags = gradient_ascent(dbi, d,step, iterations)

n = 3